In [23]:
import os
import yaml
import csv
import pickle
import pandas as pd
from collections import defaultdict

import googleapiclient.discovery

def dataset_from_csv(path_to_csv):
    if path_to_csv[-4:] != '.csv':
        path_to_csv += '.csv'
    df = pd.read_csv(path_to_csv)  
    return df

In [24]:
dummy_df = dataset_from_csv("dummy_dataset")
broken_df = dataset_from_csv("broken_dataset")

In [25]:
import string

def create_youtube_API(path_to_config):
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    
    # Load YAML config file
    try:
        config = yaml.safe_load(open(path_to_config))
    except :
        raise Exception(f"Could not load yaml at {path_to_config}")

    api_service_name = "youtube"
    api_version = "v3"
    DEVELOPER_KEY = config['api_key']

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)    
    
    return youtube

def comments_from_video_ids(youtube, video_ids):
    comments_dict = defaultdict(list)
    punctuation = string.punctuation
    punctuation += '’'
    
    for video_id in video_ids:
        request = youtube.commentThreads().list(
            part="snippet",
            maxResults=100,
            order="relevance",
            videoId=video_id
        )
        response = request.execute()  
        
        for item in response["items"]:  
            top_level_comment = item["snippet"]["topLevelComment"]
            comment_text = top_level_comment['snippet']['textOriginal']
            
            # Remove punctuation
            comment_text = comment_text.translate(str.maketrans('', '', punctuation))

            # Lowercasing
            comment_text = comment_text.lower()
            comments_dict[video_id].append(comment_text)
            
    return comments_dict

In [26]:
youtube = create_youtube_API("config.yml")
comments_dict = comments_from_video_ids(youtube, dummy_df['video_id'])

In [27]:
key = list(comments_dict.keys())[2]
print(comments_dict[key])

['watching matpats descent into madness in trying to solve or make a narrative sense of this franchise is a joy honestly', 'matpats dedication to this series is unlike no other mad respect to mat for raising a generation of fnaf nerds 😭', 'i think what actually happens is the devs dont have actual answers but just like creating mystery and twists with each game but mattpats full timeline fits all the major pieces so well that they should just adopt it as canon and build from there imo', 'a huge shoutout to the editors behind the magic and of course the theorists who all compiled their found knowledge into this cohesive work of art thanks for the wild ride everyone at this point game theory better be apart of the fnaf world somehow theyve poured so much into this series for us', 'were proud of you too matpat how you keep track of everything in this franchise is absolutely beyond me', 'matpat is actually surprisingly poetic while describing the settings looks and smells of the fazbear un

[nltk_data] Downloading package stopwords to C:\Users\Tony
[nltk_data]     Lay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


i think what actually happens is the devs dont have actual answers but just like creating mystery and twists with each game but mattpats full timeline fits all the major pieces so well that they should just adopt it as canon and build from there imo
